In [1]:
!pip install psycopg2

In [2]:
#conn.close()# close the connection

In [3]:
import psycopg2

In [4]:
conn = psycopg2.connect(
    host="va-jde-cx-vdas-dev-v1-redshiftcluster-6mripk5gz2a0.csicjjfzkah1.us-east-1.redshift.amazonaws.com",
    database="cx",
    user="cxuser_ajay",
    password="Verisk@123",
    port=5439)

In [5]:
cursor = conn.cursor()


In [39]:
cursor.execute(""" select distinct * from (
select  a."name" ,cpep.user_id,cpep.session_id,cpep.event_properties_productname,extract(month from cpep.event_time::date) as month_
,extract(year from cpep.event_time::date) as year_ from account1 a 
left join contacts1 c on c.accountid =a.x18_digit_account_id__c 
left join woodmac_amplitude.client_portal_evolve_prod_225035 cpep on cpep.user_id =c.entitlementidentityid__c 
where cpep.event_properties_productname !='' and cpep.event_properties_productname is not null
and cpep.server_upload_time::date<='2020-11-30')limit 0""")
colnames = [desc[0] for desc in cursor.description]


In [40]:

import pandas as pd
colnames

['name',
 'user_id',
 'session_id',
 'event_properties_productname',
 'month_',
 'year_']

In [41]:
cursor.execute("""select distinct * from (
select  a."name" ,cpep.user_id,cpep.session_id,cpep.event_properties_productname,extract(month from cpep.event_time::date) as month_
,extract(year from cpep.event_time::date) as year_
from account1 a 
left join contacts1 c on c.accountid =a.x18_digit_account_id__c 
left join woodmac_amplitude.client_portal_evolve_prod_225035 cpep on cpep.user_id =c.entitlementidentityid__c 
where cpep.event_properties_productname !='' and cpep.event_properties_productname is not null
and cpep.server_upload_time::date<='2020-11-30' 
and a.x18_digit_account_id__c in (select distinct x18_digit_account_id__c from (
select  a.x18_digit_account_id__c,a."name" ,max(ss.sbqq__enddate__c) 
,case 
     when max(ss.sbqq__enddate__c)>='2020-12-01' then 'active' else 'inactive' 
     end as status_of_account
from account1 a 
left join sbqq_subscription1 ss on ss.sbqq__account__c =a.x18_digit_account_id__c 
left join sbqq_quoteline1 sq on sq.id =ss.sbqq__quoteline__c 
left join order_class1 oc on oc.intorderclassid__c =sq.lineitemorderclassid__c 
where oc.strsector__c in ('Metals','Coal')  and 
a.sell_serve_categorisation__c in ('NCD','Local','Long-tail','Basic')
group by a.x18_digit_account_id__c ,a."name"
having status_of_account ='active'))) """)
rows = cursor.fetchall()

In [42]:
amplitude=pd.DataFrame(rows,columns=colnames)

In [43]:
amplitude.head()

,name,user_id,session_id,event_properties_productname,month_,year_
0,East Penn,00u7zy581yqTKbqn42p7,1603300189789,Base Metals Markets Tool - content:wmportal,10,2020
1,OECD,00u7fugjyiLrnhiru2p7,1603310655489,Steel Markets Tool - content:wmportal,10,2020
2,Minera Chinalco Perú S.A.,00u7zy4sblPjo4ZGQ2p7,1603307607855,Base Metals Markets Tool - content:wmportal,10,2020
3,Cheniere,00u7fufqqrb2jOo842p7,1603316369902,The Edge - content:wmportal,10,2020
4,NATIXIS,00u7fuzwoyxRx6YaN2p7,1603356604567,Upstream Service - content:wmportal,10,2020


In [44]:
import pandas as pd

In [45]:
prod = amplitude['event_properties_productname'].str.split(',').map(len)

In [46]:
amplitude.head()

,name,user_id,session_id,event_properties_productname,month_,year_
0,East Penn,00u7zy581yqTKbqn42p7,1603300189789,Base Metals Markets Tool - content:wmportal,10,2020
1,OECD,00u7fugjyiLrnhiru2p7,1603310655489,Steel Markets Tool - content:wmportal,10,2020
2,Minera Chinalco Perú S.A.,00u7zy4sblPjo4ZGQ2p7,1603307607855,Base Metals Markets Tool - content:wmportal,10,2020
3,Cheniere,00u7fufqqrb2jOo842p7,1603316369902,The Edge - content:wmportal,10,2020
4,NATIXIS,00u7fuzwoyxRx6YaN2p7,1603356604567,Upstream Service - content:wmportal,10,2020


In [47]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import chain

In [48]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens = amplitude['event_properties_productname'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
amplitude_evolve = pd.DataFrame({'user_id': np.repeat(amplitude['user_id'], lens),
                        'session_id': np.repeat(amplitude['session_id'], lens),
                        'name': np.repeat(amplitude['name'], lens),
                        'month_': np.repeat(amplitude['month_'], lens),
                        'year_': np.repeat(amplitude['year_'], lens),
                    'event_properties_productname': chainer(amplitude['event_properties_productname'])
                    })

In [49]:
amplitude_evolve[amplitude_evolve['name']=='TNB Fuel Services']

,user_id,session_id,name,month_,year_,event_properties_productname
491,00u7fv0dawkcqdBmr2p7,1604470606555,TNB Fuel Services,11,2020,Coal Market Service - content:wmportal
1224,00u7fun65rC3eTqnp2p7,1604301780258,TNB Fuel Services,11,2020,Coal Market Service - content:wmportal
1423,00u7fv0dawkcqdBmr2p7,1605052372263,TNB Fuel Services,11,2020,Coal Market Service - content:wmportal
2762,00u7fun65rC3eTqnp2p7,1604120605698,TNB Fuel Services,10,2020,Base Metals Markets Tool - content:wmportal
2925,00u7fun65rC3eTqnp2p7,1606145174410,TNB Fuel Services,11,2020,Coal Market Service - content:wmportal
2982,00u7fv0dawkcqdBmr2p7,1605879081648,TNB Fuel Services,11,2020,The Edge - content:wmportal
3103,00u7fv0dawkcqdBmr2p7,1605879081648,TNB Fuel Services,11,2020,Base Metals Markets Tool - content:wmportal
3103,00u7fv0dawkcqdBmr2p7,1605879081648,TNB Fuel Services,11,2020,Base Metals Markets Tool - content:wmportal
3116,00u7fv0dawkcqdBmr2p7,1605924265732,TNB Fuel Services,11,2020,The Edge - content:wmportal
3318,00u7fv0dawkcqdBmr2p7,1604463703509,TNB Fuel Services,11,2020,Coal Market Service - content:wmportal


In [50]:
amplitude_evolve.groupby('event_properties_productname')['user_id'].nunique().sort_values(ascending=False).to_csv('Product_name_amplitude.csv',index=False,header=True)

In [19]:
amplitude_evolve['event_properties_productname'].to_csv('Product_name_amplitude.csv',index=False,header=True)

In [51]:
amplitude_evolve.to_csv('amplitude_evolve.csv',index=False,header=True)

In [38]:
cd

C:\Users\Verisk


In [124]:
amplitude_evolve.head()

,accountid,name,event_properties_productid
0,001D000000lYiMfIAK,Barclays Capital,9203b43e-12d0-427f-a5d7-40eac1eb8648
1,001D000000lYiY2IAK,BNP Paribas,eda04da1-6bd1-414b-9119-099cdb9cc470
2,001D000000lYisqIAC,Chevron Corporation,ed6268d9-7f5c-4bce-a36e-d06c663a61c1
3,001D000000lYisqIAC,Chevron Corporation,ed6268d9-7f5c-4bce-a36e-d06c663a61c1
3,001D000000lYisqIAC,Chevron Corporation,8c3d074c-5658-4831-a1eb-b5f2de7ce20f


In [149]:
amplitude_evolve[amplitude_evolve['name']=='DnB Nor']

,accountid,name,event_properties_productid
207,001D000000lYjHdIAK,DnB Nor,a5e4c2d4-7e54-4f0a-be84-86251be810bc
308,001D000000lYjHdIAK,DnB Nor,a5e4c2d4-7e54-4f0a-be84-86251be810bc
308,001D000000lYjHdIAK,DnB Nor,a5e4c2d4-7e54-4f0a-be84-86251be810bc
648,001D000000lYjHdIAK,DnB Nor,a5e4c2d4-7e54-4f0a-be84-86251be810bc
1331,001D000000lYjHdIAK,DnB Nor,a5e4c2d4-7e54-4f0a-be84-86251be810bc
...,...,...,...
18734,001D000000lYjHdIAK,DnB Nor,a5e4c2d4-7e54-4f0a-be84-86251be810bc
18734,001D000000lYjHdIAK,DnB Nor,a5e4c2d4-7e54-4f0a-be84-86251be810bc
18867,001D000000lYjHdIAK,DnB Nor,a5e4c2d4-7e54-4f0a-be84-86251be810bc
18867,001D000000lYjHdIAK,DnB Nor,a5e4c2d4-7e54-4f0a-be84-86251be810bc


In [15]:
import psycopg2



conn = psycopg2.connect(
    host="va-jde-cx-vdas-dev-v1-redshiftcluster-6mripk5gz2a0.csicjjfzkah1.us-east-1.redshift.amazonaws.com",
    database="cx",
    user="cxuser_ajay",
    password="Verisk@123",
    port=5439)

In [14]:
conn.close()

In [16]:
cursor = conn.cursor()

In [21]:
cursor.execute(""" select  distinct cpep2.session_id , cpep2.user_id,cpep2.server_upload_time::date ,cpep2.event_time::Date,cpep2.event_properties_productname,mcc."name" 
from woodmac_salesforce.account mcc
left join woodmac_salesforce.contact c on c.accountid = mcc.x18_digit_account_id__c
left join woodmac_amplitude.client_portal_evolve_prod_225035 cpep2 on cpep2.user_id =c.entitlementidentityid__c
where  mcc.sell_serve_categorisation__c in ('NCD','Local','Long-tail','Basic')
and cpep2.server_upload_time::date<='2021-02-24' 
and extract(month from cpep2.event_time::date)=2
and extract(year from cpep2.event_time::date)=2021
and mcc.x18_digit_account_id__c in   (select distinct x18_digit_account_id__c from (
select  a.x18_digit_account_id__c,a."name" ,max(ss.sbqq__enddate__c) 
,case 
     when max(ss.sbqq__enddate__c)>=current_date then 'active' else 'inactive' 
     end as status_of_account
from woodmac_salesforce.account a 
left join woodmac_salesforce.sbqq_subscription ss on ss.sbqq__account__c =a.x18_digit_account_id__c 
left join woodmac_salesforce.sbqq_quoteline sq on sq.id =ss.sbqq__quoteline__c 
left join woodmac_salesforce.order_class oc on oc.intorderclassid__c =sq.lineitemorderclassid__c 
where oc.strsector__c in ('Metals','Coal') and  
a.sell_serve_categorisation__c in ('NCD','Local','Long-tail','Basic')
group by a.x18_digit_account_id__c ,a."name"
))

   limit 0""")

colnames = [desc[0] for desc in cursor.description]


colnames

cursor.execute(""" select  distinct cpep2.session_id , cpep2.user_id ,cpep2.server_upload_time::date,cpep2.event_time::Date,cpep2.event_properties_productname,mcc."name" 
from woodmac_salesforce.account mcc
left join woodmac_salesforce.contact c on c.accountid = mcc.x18_digit_account_id__c
left join woodmac_amplitude.client_portal_evolve_prod_225035 cpep2 on cpep2.user_id =c.entitlementidentityid__c
where  mcc.sell_serve_categorisation__c in ('NCD','Local','Long-tail','Basic')
and cpep2.server_upload_time::date<='2021-02-24' 
and extract(month from cpep2.event_time::date)=2
and extract(year from cpep2.event_time::date)=2021
and mcc.x18_digit_account_id__c in   (select distinct x18_digit_account_id__c from (
select  a.x18_digit_account_id__c,a."name" ,max(ss.sbqq__enddate__c) 
,case 
     when max(ss.sbqq__enddate__c)>=current_date then 'active' else 'inactive' 
     end as status_of_account
from woodmac_salesforce.account a 
left join woodmac_salesforce.sbqq_subscription ss on ss.sbqq__account__c =a.x18_digit_account_id__c 
left join woodmac_salesforce.sbqq_quoteline sq on sq.id =ss.sbqq__quoteline__c 
left join woodmac_salesforce.order_class oc on oc.intorderclassid__c =sq.lineitemorderclassid__c 
where oc.strsector__c in ('Metals','Coal') and  
a.sell_serve_categorisation__c in ('NCD','Local','Long-tail','Basic')
group by a.x18_digit_account_id__c ,a."name"
))

  """)

rows = cursor.fetchall()

In [22]:
import pandas as pd

In [23]:
amplitude=pd.DataFrame(rows,columns=colnames)

amplitude.head(4)

,session_id,user_id,server_upload_time,event_time,event_properties_productname,name
0,1612882744027,00u7fuqglwen9U6pd2p7,2021-02-09,2021-02-09,,American Electric Power
1,1613076303460,00u7fuqglwen9U6pd2p7,2021-02-11,2021-02-11,,American Electric Power
2,1613660849959,00u7fuqglwen9U6pd2p7,2021-02-18,2021-02-18,,American Electric Power
3,1612139735652,00u7fv1b5n4tp7T0J2p7,2021-02-01,2021-02-01,,Furukawa Metals & Resources


In [24]:
amplitude.to_csv('FebDataM&Maccounts.csv')